In [75]:
import warnings
warnings.filterwarnings('ignore')

In [76]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [77]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier # doctest: +NORMALIZE_WHITESPACE

# Read the CSV and Perform Basic Data Cleaning

In [78]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [79]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

df.replace(['w', 'f'], np.nan, inplace=True)

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [80]:
# Binary encoding using Pandas (single column)
df2 = df.copy()
df2_binary_encoded = pd.get_dummies(df2, columns=["home_ownership", "loan_status","verification_status", "pymnt_plan", "hardship_flag","debt_settlement_flag"])
df2_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_OWN,home_ownership_RENT,loan_status_high_risk,loan_status_low_risk,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,Mar-2019,27.24,0.0,0.0,8.0,0.0,...,0,1,0,1,0,1,0,1,1,1
1,25000.0,0.2000,929.09,105000.0,Mar-2019,20.23,0.0,0.0,17.0,1.0,...,0,0,0,1,0,0,1,1,1,1
2,20000.0,0.2000,529.88,56000.0,Mar-2019,24.26,0.0,0.0,8.0,0.0,...,0,0,0,1,0,0,1,1,1,1
3,10000.0,0.1640,353.55,92000.0,Mar-2019,31.44,0.0,1.0,10.0,1.0,...,0,1,0,1,0,0,1,1,1,1
4,22000.0,0.1474,520.39,52000.0,Mar-2019,18.76,0.0,1.0,14.0,0.0,...,0,0,0,1,1,0,0,1,1,1


In [81]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df3 = df2.copy()
df3['home_ownership'] = le.fit_transform(df2['home_ownership']) 
df3['loan_status'] = le.fit_transform(df2['loan_status']) 
df3['verification_status'] = le.fit_transform(df2['verification_status']) 
df3['pymnt_plan'] = le.fit_transform(df2['pymnt_plan']) 
df3['hardship_flag'] = le.fit_transform(df2['hardship_flag']) 
df3['debt_settlement_flag'] = le.fit_transform(df2['debt_settlement_flag']) 
df3.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,Mar-2019,1,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,Mar-2019,1,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,Mar-2019,1,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,Mar-2019,1,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,Mar-2019,1,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


# Ensemble Learners

In this section, we compared two ensemble algorithms to determine which algorithm results in the best performance. We trained a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, we completed the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: We used a random state of 1 for each algorithm to ensure consistency between tests

In [82]:
# Define the features set.
X = df3.copy()
X = X.drop("loan_status", axis=1)
X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,Mar-2019,0,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,Mar-2019,0,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,Mar-2019,0,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,Mar-2019,0,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,Mar-2019,0,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


In [83]:
# Define the target set.
y = df3["loan_status"].ravel()
y[:5]

array([1, 1, 1, 1, 1])

In [84]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [85]:
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({1: 68470, 0: 347})


In [86]:
X, y = make_classification(n_classes=2, class_sep=2,
                           weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,
                           n_features=20, n_clusters_per_class=1, n_samples=100, random_state=1)

In [87]:
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({1: 90, 0: 10})


# Balanced Random Forest Classifier

## Train the model using the training data

In [88]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({1: 68, 0: 7})

In [89]:
# Splitting into Train and Test sets.
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

In [90]:
brf.fit(X_train, y_train) 

BalancedRandomForestClassifier(random_state=1)

In [91]:
y_pred = brf.predict(X_test)

## Print the confusion matrix from sklearn.metrics

In [92]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = brf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 3,  0],
       [ 1, 21]], dtype=int64)

## Calculate the balanced accuracy score from sklearn.metrics

In [93]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

0.9772727272727273

## Generate a classication report using the report from imbalanced-learn

In [94]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      1.00      0.95      0.86      0.98      0.96         3
          1       1.00      0.95      1.00      0.98      0.98      0.95        22

avg / total       0.97      0.96      0.99      0.96      0.98      0.95        25



In [95]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,3,0
Actual 1,1,21


In [96]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,3,0
Actual 1,1,21


Accuracy Score : 0.9772727272727273
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      1.00      0.95      0.86      0.98      0.96         3
          1       1.00      0.95      1.00      0.98      0.98      0.95        22

avg / total       0.97      0.96      0.99      0.96      0.98      0.95        25



## Print the feature importance sorted in descending order along with the feature score

In [97]:
brf.feature_importances_ 

array([0.04431337, 0.03603965, 0.01791988, 0.23500298, 0.005875  ,
       0.01959636, 0.03503914, 0.17839244, 0.0437    , 0.03283954,
       0.0175303 , 0.0310451 , 0.03077743, 0.03138095, 0.03264372,
       0.04557458, 0.0526723 , 0.06194108, 0.01785185, 0.02986432])

In [98]:
# We can sort the features by their importance.
importances = list(zip(brf.feature_importances_, df.columns))
importances.sort(reverse=True)
importances

[(0.23500298220298219, 'home_ownership'),
 (0.17839244199363252, 'loan_status'),
 (0.06194107744107744, 'out_prncp'),
 (0.052672298953251345, 'initial_list_status'),
 (0.045574579124579136, 'total_acc'),
 (0.044313374546329096, 'loan_amnt'),
 (0.04370000000000001, 'pymnt_plan'),
 (0.036039646035181755, 'int_rate'),
 (0.03503914141414142, 'issue_d'),
 (0.03283954124579125, 'dti'),
 (0.032643722943722954, 'revol_bal'),
 (0.031380952380952384, 'pub_rec'),
 (0.03104509952129, 'inq_last_6mths'),
 (0.03077742832364045, 'open_acc'),
 (0.029864323507180653, 'total_pymnt'),
 (0.019596356421356424, 'verification_status'),
 (0.017919879062736208, 'installment'),
 (0.017851851851851855, 'out_prncp_inv'),
 (0.01753030303030303, 'delinq_2yrs'),
 (0.005875000000000003, 'annual_inc')]

## Models’ performance  
The results show that:
- Out of 3 good loan applications (Actual 0), 3 were predicted to be good (Predicted 0), which we call true positives.
- Out of 3 good loan applications (Actual 0), 0 were predicted to be bad (Predicted 1), which are considered false negatives.
- Out of 22 bad loan applications (Actual 1), 0 was predicted to be good (Predicted 0) and are considered false positives.
- Out of 22 bad loan applications (Actual 1), 22 were predicted to be bad (Predicted 1) and are considered true negatives.  

### **Precision**  
Precision is the measure of how reliable a positive classification is. From our results, the precision for the good loan applications can be determined by the ratio TP/(TP + FP), which is **3/(3 + 1) = 0.75**. The precision for the bad loan applications can be determined as follows: **22/(22 + 0) = 1**. A low precision is indicative of a large number of false positives—of the **21** loan applications we predicted to be bad applications, **0** were actually good loan applications.  
### **Recall scores**  
Recall is the ability of the classifier to find all the positive samples. It can be determined by the ratio: TP/(TP + FN), or **3/(3 + 0) = 1** for the good loans and **21/(21 + 1) = 0.9545** for the bad loans. A low recall is indicative of a large number of false negatives.  
### **Balanced accuracy score**  
An accuracy score is not always an appropriate or a meaningful performance metric. This program’s accuracy score appears to be high at **0.9772727272727273**.  
### **F1 score**  
F1 score is a weighted average of the true positive rate (recall) and precision, where the best score is **1.0** and the worst is **0.0**.  
### **Support**  
Support is the number of actual occurrences of the class in the specified dataset. For our results, there are **3** actual occurrences for the good loans and **22** actual occurrences for bad loans.  
### **Recommendation on the model to use:**  
In summary, this model may **not** be a good one for preventing fraudulent loan applications. The model's absence of prediction samples and precision makes it **not** good enough to state that the model will be good at classifying fraudulent loan applications.

# Easy Ensemble AdaBoost Classifier

In [99]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({1: 68, 0: 7})

## Train the model using the training data

In [100]:
X, y = make_classification(n_classes=2, class_sep=2,
                           weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,
                           n_features=20, n_clusters_per_class=1, n_samples=100, random_state=1)

In [101]:
print('Original dataset shape %s' % Counter(y)) 

Original dataset shape Counter({1: 90, 0: 10})


In [102]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [103]:
eec = EasyEnsembleClassifier(random_state=1)

In [104]:
eec.fit(X_train, y_train) # doctest: +ELLIPSIS

EasyEnsembleClassifier(random_state=1)

In [105]:
y_pred = eec.predict(X_test)

## Print the confusion matrix from sklearn.metrics

In [106]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = eec.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 3,  0],
       [ 0, 22]], dtype=int64)

## Calculate the balanced accuracy score from sklearn.metrics

In [107]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

1.0

## Generate a classication report using the report from imbalanced-learn

In [108]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00         3
          1       1.00      1.00      1.00      1.00      1.00      1.00        22

avg / total       1.00      1.00      1.00      1.00      1.00      1.00        25



In [109]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,3,0
Actual 1,0,22


In [110]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,3,0
Actual 1,0,22


Accuracy Score : 1.0
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00         3
          1       1.00      1.00      1.00      1.00      1.00      1.00        22

avg / total       1.00      1.00      1.00      1.00      1.00      1.00        25



## Models’ performance  
The results show that:
- Out of 3 good loan applications (Actual 0), 3 were predicted to be good (Predicted 0), which we call true positives.
- Out of 3 good loan applications (Actual 0), 0 were predicted to be bad (Predicted 1), which are considered false negatives.
- Out of 22 bad loan applications (Actual 1), 0 was predicted to be good (Predicted 0) and are considered false positives.
- Out of 22 bad loan applications (Actual 1), 22 were predicted to be bad (Predicted 1) and are considered true negatives.  

### **Precision**  
Precision is the measure of how reliable a positive classification is. From our results, the precision for the good loan applications
can be determined by the ratio TP/(TP + FP), which is **3/(3 + 0) = 1**. The precision for the bad loan applications can be
determined as follows: **22/(22 + 0) = 1**. A low precision is indicative of a large number of false positives—of the **22** loan applications we predicted to be bad applications, **0** were actually good loan applications.  
### **Recall scores**  
Recall is the ability of the classifier to find all the positive samples. It can be determined by the ratio: TP/(TP + FN), or **3/(3 + 0) = 1** for the good loans and **22/(22 + 0) = 1** for the bad loans. A low recall is indicative of a large number of false negatives.  
### **Balanced accuracy score**  
An accuracy score is not always an appropriate or a meaningful performance metric. This program’s accuracy score appears to be perfect at **100%**.  
### **F1 score**  
F1 score is a weighted average of the true positive rate (recall) and precision, where the best score is **1.0** and the worst is **0.0**.  
### **Support**  
Support is the number of actual occurrences of the class in the specified dataset. For our results, there are **3** actual occurrences for the good loans and **22** actual occurrences for bad loans.  
### **Recommendation on the model to use:**  
In summary, this model may be the best one for preventing fraudulent loan applications. Although, the model's absence of prediction samples makes it **not** good enough to state that the model will be good at classifying fraudulent loan applications.